**19장 – 대규모 텐서플로 모델 훈련과 배포**

# 개요

1. **모델의 실제 제품화**  
   - 모델을 실제 제품에 장착하려면 시스템의 실시간 데이터에 적용해야 하며, 이를 위해 모델을 웹 서비스화해야 함.
   - REST API를 사용해 시스템에서 언제든지 모델에 쿼리를 보낼 수 있음.
   - 새로운 데이터를 바탕으로 모델을 정기적으로 재훈련하고 업데이트된 버전을 반영해야 함.
   - 모델 버전 관리 및 A/B 테스트를 통해 여러 모델을 동시에 실행하거나 문제 발생 시 롤백이 가능해야 함.

2. **서비스 확장과 안정성 확보**  
   - 많은 QPS(초당 쿼리 수)를 처리하기 위해 서비스 규모를 확장해야 함.
   - 구글 버텍스 AI 플랫폼이나 TF Serving을 활용해 효율적이고 안정적으로 모델 서비스 가능.
   - 클라우드 플랫폼을 통해 모니터링 도구 등 부가 기능 활용 가능.

3. **훈련 속도와 실험의 중요성**  
   - 훈련 데이터가 많거나 복잡한 모델은 훈련 시간이 길어질 수 있음.
   - 빠른 훈련과 실험을 위해 GPU/TPU와 같은 하드웨어 가속기와 분산 전략 API 사용.
   - 새로운 아이디어를 실험하기 위해 훈련 속도는 매우 중요.

4. **구체적인 학습과 배포 전략**  
   - TF 서빙 및 구글 버텍스 AI 플랫폼을 활용해 모델 배포 방법 학습.
   - GPU 및 분산 전략을 사용해 훈련 속도 향상.
   - 버텍스 AI로 대규모 모델 훈련 및 하이퍼파라미터 튜닝 방법 학습 예정.

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/rickiepark/handson-ml3/blob/main/19_training_and_deploying_at_scale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# 설정

이 프로젝트에는 Python 3.7 이상이 필요합니다:

In [1]:
import sys

assert sys.version_info >= (3, 7)

그리고 TensorFlow ≥ 2.8:

In [2]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

2024-11-30 09:14:36.494242: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-30 09:14:36.494269: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-30 09:14:36.494293: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 09:14:36.499042: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-30 09:14:37.033368: W tensorflow/compiler/

코랩에서 실행하는 경우, 이 노트북의 뒷부분에서 사용하게 될 구글 AI 플랫폼 클라이언트 라이브러리를 설치해야 합니다. 버전 비호환성에 대한 경고는 무시해도 됩니다.

* **경고**: 코랩에서는 설치 후 런타임을 다시 시작하고 다음 셀을 계속 진행해야 합니다.

In [3]:
import sys
if "google.colab" in sys.modules or "kaggle_secrets" in sys.modules:
    %pip install -q -U google-cloud-aiplatform

이 장에서는 하나 이상의 GPU에서 모델을 실행하거나 훈련하는 방법에 대해 설명하므로 적어도 하나 이상의 GPU가 있는지 확인하거나 그렇지 않으면 경고를 발행합니다:

In [4]:
if not tf.config.list_physical_devices('GPU'):
    print("GPU가 감지되지 않았습니다. 신경망은 GPU가 없으면 매우 느릴 수 있습니다.")
    if "google.colab" in sys.modules:
        print("런타임 > 런타임 유형 변경으로 이동하여 하드웨어 가속기로 GPU를 선택합니다.")

2024-11-30 09:14:39.521756: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-30 09:14:39.542195: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-30 09:14:39.542374: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

# 텐서플로 모델 서빙하기

1. **모델 서빙의 필요성**  
   - 텐서플로로 학습된 모델은 일반적으로 `predict()` 메서드 호출만으로 사용할 수 있습니다.  
   - 그러나 시스템 규모가 커지면서 모델을 별도의 서비스로 감싸는 것이 바람직한 경우가 생깁니다.  
   - 이러한 서비스는 예측 요청(REST/gRPC API 등)을 처리하며, 모델과 나머지 시스템을 분리하는 역할을 합니다.

2. **모델 서빙의 장점**  
   - **독립성**: 모델을 나머지 시스템과 분리하여 운영.  
   - **유연성**: 모델 버전 변경이 용이하며, 필요에 따라 서비스 규모를 확장 가능.  
   - **테스트와 개발 단순화**: A/B 테스트 실행과 동일한 모델 버전 유지가 수월.  

3. **기술 구성**  
   - 모델 서빙에는 다양한 기술이 활용됩니다(예: Flask와 같은 웹 프레임워크).  
   - 텐서플로 자체 서빙 도구(TensorFlow Serving)를 이용하면 별도로 구현할 필요가 없습니다.

먼저 TF 서빙을 사용하여 모델을 배포한 다음 Google Vertex AI에 배포해 보겠습니다.

## 텐서플로 서빙 사용하기

**TF 서빙의 기술적 특징**
- C++로 작성되어 높은 성능과 효율성을 제공.
- **주요 기능**:
  - 높은 부하 처리 능력.
  - 여러 모델 서비스 가능.
  - 모델 저장소에서 최신 버전을 자동 배포.
- TensorFlow 모델을 `SavedModel` 포맷으로 변환하여 서빙에 적합하게 준비해야 함.

<img src="./images/fig_19_01.png" width="800">

### SavedModel 내보내기

- 모델 저장은 `model.save()` 메서드로 간단히 수행 가능.

In [7]:
from pathlib import Path
import tensorflow as tf

In [18]:
from pathlib import Path
import tensorflow as tf

# MNIST 데이터셋을 로드하고 학습/검증 데이터로 분할
# load_data()를 통해 학습용과 테스트용 데이터를 받아옴
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist

# 전체 학습 데이터에서 앞의 5000개를 검증 데이터로 분리
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# 재현성을 위해 랜덤 시드 설정 및 이전 모델 초기화
tf.random.set_seed(42)
tf.keras.backend.clear_session()

# Sequential API를 사용하여 간단한 MNIST 분류 모델 구축
model = tf.keras.Sequential([
    # 28x28 이미지를 1차원으로 펼침, uint8 입력을 받음
    tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
    # 픽셀값을 0-1 범위로 정규화
    tf.keras.layers.Rescaling(scale=1 / 255),
    # 100개 뉴런을 가진 은닉층, ReLU 활성화 함수 사용
    tf.keras.layers.Dense(100, activation="relu"),
    # 10개 뉴런을 가진 출력층(0-9 숫자), softmax 활성화 함수 사용
    tf.keras.layers.Dense(10, activation="softmax")
])

# 모델 컴파일: 손실 함수, 옵티마이저, 평가 지표 설정
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])



2024-11-30 09:19:05.926501: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-30 09:19:05.926688: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-30 09:19:05.926819: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
# 모델 학습: 10 에포크 동안 학습하며 검증 데이터로 성능 모니터링
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

In [8]:
# 학습된 모델을 SavedModel 형식으로 저장
model_name = "my_mnist_model"
model_version = "0001"
model_path = Path(model_name) / model_version
# model.save(model_path, save_format="tf")

- 일반적으로 내보낸 최종 모델에 모든 전처리 층을 포함하는 것이 좋습니다. 
- 이렇게 하면 제품으로 배포했을 때 원래 형태 그대로 데이터를 주입할 수 있습니다. 
- 또 모델을 사용하는 애플리케이션 내에서 전처리를 별도로 관리할 필요가 없습니다. 
- 모델 안에서 전처리 단계를 처리하면 나중에 모델을 업데이트하기가 훨씬 수월하고 모델과 필요한 전처리 단계가 맞지 않는 문제를 피할 수 있습니다.

>**주의사항** : SavedModel이 계산 그래프를 저장하므로 임의의 파이썬 코드를 갖는 tf.py_function() 연산을 제외한 텐서플로 연산만을 사용한 모델에 적용 가능

파일 트리를 살펴보겠습니다(각 파일의 용도에 대해서는 10장에서 설명했습니다):

In [9]:
sorted([str(path) for path in model_path.parent.glob("**/*")])  # 추가 코드

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/keras_metadata.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index']

SavedModel을 검사해 보겠습니다:

In [10]:
# saved_model_cli를 사용하여 저장된 모델의 기본 정보를 표시합니다
!saved_model_cli show --dir '{model_path}'

2024-11-30 09:16:11.443117: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-30 09:16:11.443156: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-30 09:16:11.443180: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 09:16:11.447674: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-30 09:16:11.976206: W tensorflow/compiler/

출력 내용: SavedModel에 포함된 **태그 세트(tag-sets)** 확인 가능 (예: `'serve'`).

- **메타그래프(MetaGraph)**:
  - 계산 그래프와 **입력/출력 정보, 타입, 크기**를 포함.
  - 각 메타그래프는 하나 이상의 태그(tag)로 구분.
  - 예시 태그:
    - `'train'`: 훈련 연산 포함.
    - `'serve'`: 예측 연산 포함 (서빙에 사용).
    - `'gpu'`: GPU 관련 연산 포함.
- **태그의 역할**:
  - 특정 태그를 지정해 해당 메타그래프만 선택적으로 사용할 수 있음.
  - 포함된 SignatureDef 키 목록 확인 (`serving_default`, `__saved_model_init_op` 등).


In [11]:
!saved_model_cli show --dir '{model_path}' --tag_set serve

2024-11-30 09:16:21.274769: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-30 09:16:21.274800: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-30 09:16:21.274823: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 09:16:21.279306: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-30 09:16:21.809311: W tensorflow/compiler/

 **SignatureDef 정의**
- **SignatureDef**: 모델의 입력과 출력 서명을 정의하는 텐서플로 구성 요소.
- 주요 서명:
  - **`__saved_model_init_op`**: 초기화 함수 (대부분 신경 쓰지 않아도 됨).
  - **`serving_default`**: 기본 서명으로, 모델 저장 시 자동 생성.


In [12]:
!saved_model_cli show --dir '{model_path}' --tag_set serve \
                      --signature_def serving_default

2024-11-30 09:16:26.354583: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-30 09:16:26.354612: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-30 09:16:26.354635: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 09:16:26.359087: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-30 09:16:26.881159: W tensorflow/compiler/

- 출력:
  - **입력 (Input)**:
    - 이름: `flatten_input`
    - 데이터 타입: `DT_UINT8`
    - 크기: `(-1, 28, 28)` (28x28 크기의 유연한 배치 크기)
    - 텐서 이름: `serving_default_flatten_input:0`
  - **출력 (Output)**:
    - 이름: `dense_1`
    - 데이터 타입: `DT_FLOAT`
    - 크기: `(-1, 10)` (10개의 클래스 출력)
    - 텐서 이름: `StatefulPartitionedCall:0`

더 자세한 내용을 보려면 다음 명령을 실행하세요:

```ipython
!saved_model_cli show --dir '{model_path}' --all
```

In [13]:
!saved_model_cli show --dir '{model_path}' --all

2024-11-30 09:16:30.777324: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-30 09:16:30.777358: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-30 09:16:30.777386: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 09:16:30.781937: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-30 09:16:31.316273: W tensorflow/compiler/

**사용 목적**
- **입출력 명세 확인**:
  - 모델의 예상 입력/출력 형태와 데이터 타입 확인.
- **TF Serving 연결**:
  - 이 정보를 활용하여 다음 단계에서 텐서플로 서빙을 설정.

### 텐서플로 서빙 설치하고 시작하기

이 노트북을 코랩에서 실행하는 경우, 텐서플로 서버를 설치해야 합니다:

In [10]:
if "google.colab" in sys.modules:
    url = "https://storage.googleapis.com/tensorflow-serving-apt"
    src = "stable tensorflow-model-server tensorflow-model-server-universal"
    !echo 'deb {url} {src}' > /etc/apt/sources.list.d/tensorflow-serving.list
    !curl '{url}/tensorflow-serving.release.pub.gpg' | apt-key add -
    !apt update -q && apt-get install -y tensorflow-model-server
    %pip install -q -U tensorflow-serving-api==2.11.1

`tensorflow_model_server`가 설치된 경우(예: Colab에서 이 노트북을 실행하는 경우) 다음 2개의 셀을 실행하여 서버를 시작하세요. 사용 중인 OS가 Windows인 경우, 터미널에서 `tensorflow_model_server` 명령을 실행하고 `${MODEL_DIR}`을 `my_mnist_model` 디렉터리의 전체 경로로 바꿔야 할 수 있습니다.

In [14]:
model_path.parent.absolute()

PosixPath('/mnt/mydrive/workspaces/study/ds4th_study/source/핸즈온_머신러닝/my_mnist_model')

In [15]:
import os

os.environ["MODEL_DIR"] = str(model_path.parent.absolute())

**서버 실행**
- 모델 디렉토리 경로 설정 (`MODEL_DIR`).
  - 디렉토리 예시: `my_mnist_model`
- 서버 실행 명령어:

In [16]:
%%bash --bg
tensorflow_model_server \
    --port=8500 \
    --rest_api_port=8501 \
    --model_name=my_mnist_model \
    --model_base_path="${MODEL_DIR}" >my_server.log 2>&1

- **`--rest_api_port`**: REST API 요청 수신 포트.
- **`--port`**: gRPC 요청 수신 포트.
- 실행 로그: `my_server.log`에 저장.
- 텐서플로 서빙은 **백그라운드 프로세스**로 실행되며, 지정된 경로의 모델을 로드.
- **REST 및 gRPC 요청**을 통해 모델을 배포하고 사용할 준비가 됨.
- 설치와 실행 과정은 우분투 환경에서 진행 가능하며, Docker와 같은 대안도 존재.

#### 도커 컨테이너에서 TF 서빙 실행하기

개인 컴퓨터에서 이 노트북을 실행하는 경우, 도커를 사용해 TF 서빙을 설치하려면 먼저 [Docker](https://docs.docker.com/install/)가 설치되어 있는지 확인한 후 터미널에서 다음 명령을 실행하세요. `path/to/my_mnist_model`을 `my_mnist_model` 디렉토리의 적절한 절대 경로로 대체해야 하지만, 컨테이너 경로 `/models/my_mnist_model`은 수정하지 마세요.

```bash
docker pull tensorflow/serving  # 최신 TF 서빙 이미지 다운로드

docker run -it --rm -v "/path/to/my_mnist_model:/models/my_mnist_model" \
    -p 8500:8500 -p 8501:8501 -e MODEL_NAME=my_mnist_model tensorflow/serving
```

### REST API로 TF 서빙에 쿼리하기

- REST API를 사용해 요청 데이터를 전달하기 위해 JSON 포맷 사용.
- 예시 `request_json`:

In [19]:
# json 모듈을 임포트하여 JSON 데이터를 다루기 위해 사용
import json

# 테스트 데이터셋에서 처음 3개의 이미지를 선택
X_new = X_test[:3]  # 분류할 새로운 숫자 이미지가 3개 있다고 가정합니다.

# REST API 요청을 위한 JSON 형식 데이터 생성
# signature_name: 모델의 서명 이름 지정 
# instances: 예측할 데이터를 리스트 형태로 변환
request_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [ ]:
request_json[:100] + "..." + request_json[-10:]

- Python의 `requests` 라이브러리를 사용해 HTTP POST 방식으로 전송.
- 이제 텐서플로 서빙의 REST API를 사용하여 예측을 해보겠습니다:

In [20]:
# HTTP 요청을 위한 requests 라이브러리 임포트
import requests

# TensorFlow Serving REST API 엔드포인트 URL 설정
# localhost:8501은 도커 컨테이너의 포트와 매핑됨
server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"

# POST 요청으로 데이터 전송
# request_json: 예측할 이미지 데이터가 담긴 JSON 문자열
response = requests.post(server_url, data=request_json)

# HTTP 요청이 실패하면 예외 발생
response.raise_for_status()  

# JSON 응답을 파이썬 객체로 변환
response = response.json()

In [21]:
response

{'predictions': [[4.68634571e-05,
   1.73043745e-07,
   0.000472726271,
   0.00249943556,
   7.34300272e-07,
   9.44667481e-05,
   8.48132853e-09,
   0.996759713,
   1.58201346e-05,
   0.000110146204],
  [0.000311168784,
   9.07744616e-05,
   0.980463922,
   0.00837160461,
   1.03811928e-08,
   0.000230177204,
   0.0100099817,
   6.15892268e-11,
   0.000522418355,
   3.16814464e-09],
  [2.73339283e-05,
   0.979730487,
   0.00756764784,
   0.00127741753,
   0.000332536845,
   0.000896065088,
   0.00156705291,
   0.00592685817,
   0.00247175898,
   0.000202946205]]}

In [22]:
# numpy 배열 처리를 위한 라이브러리 임포트
import numpy as np

# response의 predictions를 numpy 배열로 변환
y_proba = np.array(response["predictions"])

# 소수점 2자리까지 반올림하여 예측 확률 출력
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.98, 0.01, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

**REST API의 장단점**
- **장점**:
  - JSON 기반으로 간단하며 다양한 클라이언트에서 사용 가능.
  - 텍스트 포맷으로 표현되어 유연성 높음.
- **단점**:
  - 대규모 데이터 전송 시 효율성 낮음.
  - 숫자 데이터를 텍스트로 변환하면서 네트워크 대역폭 소모가 큼.


 **gRPC로 전환 필요성**
- 대규모 데이터 전송 및 응답 속도가 중요한 경우, **gRPC** 추천.
  - HTTP/2 기반으로 효율적이고 컴팩트한 데이터 전송 가능.

### gRPC API로 TF 서빙에 쿼리하기

- **PredictRequest** 객체를 사용하여 gRPC 요청 생성.

In [23]:
# tensorflow serving의 predict_pb2 모듈에서 PredictRequest 클래스 임포트
from tensorflow_serving.apis.predict_pb2 import PredictRequest

# 예측 요청 객체 생성
request = PredictRequest()

# 모델 스펙 설정
request.model_spec.name = model_name  # 사용할 모델 이름 지정
request.model_spec.signature_name = "serving_default"  # 기본 시그니처 이름 사용

# 입력 텐서 이름 가져오기 (첫 번째 입력 레이어의 이름)
input_name = model.input_names[0]  # flatten_input 레이어

# 입력 데이터를 텐서 프로토콜 버퍼로 변환하여 요청에 추가
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

- gRPC 채널을 사용해 요청 전송.

>스텁(Stub)은 분산 시스템에서 클라이언트와 서버 간의 통신을 단순화하는 프록시 객체입니다. gRPC 컨텍스트에서 특히 중요한 역할을 합니다. 통신 추상화, 직렬화/역직렬화 처리 등을 담당합니다.

In [24]:
# gRPC 관련 모듈 임포트
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

# localhost:8500으로 gRPC 채널 생성
channel = grpc.insecure_channel('localhost:8500')

# 예측 서비스 스텁 생성
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)

# 예측 요청을 보내고 10초 타임아웃으로 응답 받기
response = predict_service.Predict(request, timeout=10.0)

- 응답 데이터를 TensorFlow 배열로 변환.

In [25]:
response

outputs {
  key: "dense_1"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 3
      }
      dim {
        size: 10
      }
    }
    float_val: 4.6863457100698724e-05
    float_val: 1.7304374466675654e-07
    float_val: 0.0004727262712549418
    float_val: 0.0024994355626404285
    float_val: 7.343002721427183e-07
    float_val: 9.446674812352285e-05
    float_val: 8.481328528375798e-09
    float_val: 0.9967597126960754
    float_val: 1.5820134649402462e-05
    float_val: 0.00011014620395144448
    float_val: 0.0003111687838099897
    float_val: 9.077446156879887e-05
    float_val: 0.9804639220237732
    float_val: 0.008371604606509209
    float_val: 1.0381192794284289e-08
    float_val: 0.00023017720377538353
    float_val: 0.01000998169183731
    float_val: 6.158922677412804e-11
    float_val: 0.0005224183551035821
    float_val: 3.1681446355236176e-09
    float_val: 2.7333928301231936e-05
    float_val: 0.979730486869812
    float_val: 0.007567647844552994


In [26]:
# 모델의 출력 레이어 이름 가져오기
output_name = model.output_names[0]
# 응답에서 출력 텐서 프로토콜 버퍼 가져오기
outputs_proto = response.outputs[output_name]
# 텐서 프로토콜 버퍼를 텐서플로우 배열로 변환
y_proba = tf.make_ndarray(outputs_proto)

In [27]:
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.98, 0.01, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

클라이언트에 텐서플로 라이브러리가 포함되어 있지 않은 경우, 다음과 같이 응답을 넘파이 배열로 변환할 수 있습니다:

In [ ]:
# tf.make_ndarray() 대신 넘파이로 직접 변환하는 방법
# 1. 모델의 출력 레이어 이름을 가져옵니다
output_name = model.output_names[0]

# 2. 응답에서 출력 텐서 프로토콜 버퍼를 가져옵니다
outputs_proto = response.outputs[output_name]

# 3. 텐서의 차원 정보를 추출하여 shape 리스트를 만듭니다
shape = [dim.size for dim in outputs_proto.tensor_shape.dim]

# 4. float_val을 넘파이 배열로 변환하고 원래 shape으로 재구성합니다
y_proba = np.array(outputs_proto.float_val).reshape(shape)

# 5. 소수점 2자리까지 반올림하여 출력합니다
y_proba.round(2)

- **장점**:
  - REST API보다 효율적이고 빠른 데이터 전송.
  - 이진 데이터로 처리되어 네트워크 대역폭 절약.
- **보안**:
  - gRPC는 기본적으로 SSL/TLS 보안 채널 지원 가능.
  - 위 예시는 보안을 설정하지 않음 (테스트용).

### 새 모델 버전 배포하기

In [ ]:
# 추가 코드 - 새로운 MNIST 모델 버전 빌드 및 훈련
np.random.seed(42)
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
    tf.keras.layers.Rescaling(scale=1 / 255),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

- **SavedModel 포맷**으로 새로운 모델 버전을 저장.

In [ ]:
# 새로운 모델 버전 번호 지정
model_version = "0002"

# 모델을 저장할 경로 생성 (model_name/버전)
model_path = Path(model_name) / model_version

# 모델을 SavedModel 형식으로 저장
model.save(model_path, save_format="tf")

파일 트리를 다시 한 번 살펴봅시다:

In [ ]:
sorted([str(path) for path in model_path.parent.glob("**/*")])  # 추가 코드

**텐서플로 서빙에서 모델 버전 관리**
- 새로운 모델 버전을 디렉토리에 추가하면, 텐서플로 서빙은 **자동으로 버전을 탐지하고 교체**.
- 기본 동작:
  - 요청 중인 경우, 이전 버전의 모델이 응답을 마칠 때까지 유지.
  - 새 버전 준비 완료 후 요청 처리 시작.
  - 이전 버전은 안전하게 언로드(unload).

>**경고**: 텐서플로 서빙이 새 모델을 로드하기까지 잠시 기다려야 할 수 있습니다.

In [26]:
import requests

server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"

response = requests.post(server_url, data=request_json)
response.raise_for_status()
response = response.json()

In [ ]:
response.keys()

In [ ]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

 **모델 배포 중 고려 사항**
- **RAM 사용량**:
  - 새 버전 로딩 시 기존 버전과 동시 로드되므로 RAM 사용량 증가.
  - GPU 사용 시 특히 주의 필요.
- **서비스 중단 방지**:
  - 텐서플로 서빙은 요청을 모두 응답한 후 모델 전환을 완료하여 중단 방지.

**배치 처리 옵션**
- **`--enable_batching`**:
  - 요청을 일정 시간 동안 배치로 처리하여 성능 향상 가능.
  - 배치 매개변수를 설정하면 GPU 활용 극대화 가능.

**롤백 방법**
- 새 버전이 정상 작동하지 않는 경우:
  - 새로운 디렉토리 삭제(예: `my_mnist_model/0002`).

**고성능 배포를 위한 추가 설정**
- **부하가 많은 경우**:
  - 여러 서버에 텐서플로 서빙 설치 및 로드 밸런싱 사용.
  - **쿠버네티스(Kubernetes)**:
    - 다수의 컨테이너 기반 서버 관리.
    - 클라우드 서비스(AWS, GCP 등)와 연계 가능.  

<img src="./images/fig_19_02.png" width="800">

## 버텍스 AI에서 예측 서비스 만들기

**버텍스 AI(Vetex AI)란?**
- Google Cloud Platform(GCP)의 AI 플랫폼.
- 주요 기능:
  - 데이터셋 업로드 및 관리(Feature Store).
  - 자동 하이퍼파라미터 튜닝.
  - AutoML을 통한 모델 아키텍처 탐색.
  - GPU 및 TPU를 사용한 모델 학습.
  - REST/gRPC를 통해 대규모 모델 서빙.
  - Workbench로 데이터와 모델 실험 가능.

<img src="./images/fig_19_03.png" width="800">

**GCP에서 설정해야 할 준비 사항**
1. **GCP 계정 생성 및 콘솔 이동**:
   - GCP 계정이 없다면 신규 계정을 만들어야 함.

2. **GCP 무료 크레딧 제공**:
   - 신규 사용자에게 $300 크레딧 제공(90일 사용 가능).
   - 무료 체험 이후 서비스를 계속 사용하려면 결제 정보 업데이트 필요.

3. **프로젝트 생성**:
   - 모든 GCP 리소스(가상 서버, 데이터, 모델 등)는 프로젝트 단위로 관리됨.
   - GCP 콘솔에서 새 프로젝트 생성 가능.
   - 프로젝트 생성 후 결제 계정 활성화 필수.

4. **결제 계정 확인**:
   - GCP 사용 전 결제 계정이 활성화되어 있어야 함.
   - 예상 비용 확인 및 불필요한 서비스 종료로 비용 절감 가능.

5. **필요 API 활성화**:
   - Vertex AI API 및 Cloud Storage API 활성화 필요.


**버텍스 AI의 장점**
- **대규모 데이터 처리**:
  - 대량의 데이터를 효과적으로 학습 및 관리 가능.
- **통합 관리**:
  - 모델 학습, 관리, 서빙까지 GCP 플랫폼에서 원활히 수행.
- **추가 서비스**:
  - 컴퓨터 비전, 번역, 스피치-투-텍스트 등 다양한 AI API 포함.   

책의 가이드를 따라 구글 클라우드 플랫폼 계정을 생성하고 버텍스 AI 및 클라우드 스토리지 API를 활성화하세요. 그런 다음, 코랩에서 이 노트북을 실행하는 경우 다음 셀을 실행하여 구글 클라우드 플랫폼에서 사용한 것과 동일한 구글 계정으로 인증하고 코랩이 데이터에 액세스할 수 있도록 권한을 부여할 수 있습니다.

**경고: 이 노트북을 신뢰하는 경우에만 이 작업을 수행하세요!**
* https://github.com/rickiepark/handson-ml3 에 있는 공식 노트북이 아닌 경우 특히 주의하세요: 코랩 URL은 https://colab.research.google.com/github/rickiepark/handson-ml3 으로 시작합니다. 그렇지 않으면 이 코드가 여러분의 데이터로 원하는 모든 작업을 수행할 수 있습니다.

코랩에서 이 노트북을 실행하지 않는 경우, 책의 가이드를 따라 서비스 계정을 만들고 해당 서비스 계정의 키를 생성한 다음, 이 노트북의 디렉터리에 다운로드하고 이름을 `my_service_account_key.json`으로 지정해야 합니다(또는 `GOOGLE_APPLICATION_CREDENTIALS` 환경 변수가 이 파일을 가리키도록 합니다).

In [ ]:
project_id = "my_project"  ##### 이를 프로젝트 ID로 변경합니다. #####

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()
else:
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "my_service_account_key.json"

In [ ]:
from google.cloud import storage

bucket_name = "my_bucket"  ##### 고유한 버킷 이름으로 변경합니다. #####
location = "us-central1"

storage_client = storage.Client(project=project_id)
bucket = storage_client.create_bucket(bucket_name, location=location)
#bucket = storage_client.bucket(bucket_name)  # 버킷을 재사용하는 경우

In [ ]:
def upload_directory(bucket, dirpath):
    dirpath = Path(dirpath)
    for filepath in dirpath.glob("**/*"):
        if filepath.is_file():
            blob = bucket.blob(filepath.relative_to(dirpath.parent).as_posix())
            blob.upload_from_filename(filepath)

upload_directory(bucket, "my_mnist_model")

In [ ]:
# 추가 코드 – upload_directory()의 훨씬 빠른 멀티 스레드 구현
#           타깃 경로의 prefix를 받고 출력 기능도 있습니다.

from concurrent import futures

def upload_file(bucket, filepath, blob_path):
    blob = bucket.blob(blob_path)
    blob.upload_from_filename(filepath)

def upload_directory(bucket, dirpath, prefix=None, max_workers=50):
    dirpath = Path(dirpath)
    prefix = prefix or dirpath.name
    with futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_filepath = {
            executor.submit(
                upload_file,
                bucket, filepath,
                f"{prefix}/{filepath.relative_to(dirpath).as_posix()}"
            ): filepath
            for filepath in sorted(dirpath.glob("**/*"))
            if filepath.is_file()
        }
        for future in futures.as_completed(future_to_filepath):
            filepath = future_to_filepath[future]
            try:
                result = future.result()
            except Exception as ex:
                print(f"{filepath!s:60} 업로드 에러: {ex}")  # f!s is str(f)
            else:
                print(f"{filepath!s:60} 업로드 완료", end="\r")

    print(f"{dirpath!s:60} 업로드 완료")

#### 구글 클라우드 CLI와 셀

 **구글 클라우드 CLI**
- **gcloud 명령줄 인터페이스**:
  - GCP의 거의 모든 리소스를 제어 가능.
  - 포함된 도구:
    - **`gcloud`**: GCP 관리 명령어 제공.
    - **`gsutil`**: 스토리지와 상호작용 도구.
  - 예시 명령어:
    ```bash
    gcloud config list
    ```
    - 현재 설정을 표시.

- **CLI 인증**:
  - CLI에서 **`google.auth.authenticate_user()`**를 호출하여 인증.

2. **구글 클라우드 셸 (Cloud Shell)**
- **웹 브라우저 기반 환경**:
  - GCP에서 제공하는 사전 설정된 셸 환경.
  - 실행 환경:
    - 무료 리눅스 가상 머신(데비안 기반).
    - GCP SDK 사전 설치 및 설정 완료.
  - 장점:
    - 추가 인증 과정 불필요.
    - GCP 어느 곳에서나 사용 가능.

- **활성화 방법**:
  - GCP 콘솔 오른쪽 상단 **Cloud Shell 아이콘 클릭**.

3. **CLI 설치 및 초기화**
- **CLI 설치**:
  - [설치 링크](https://homl.info/gcloud) 방문.
  - 설치 후 **`gcloud init`** 명령 실행.
    - GCP 계정으로 로그인.
    - GCP 리소스에 대한 액세스 권한 부여.
    - 기본 프로젝트 및 리전(region) 선택.

<img src="./images/fig_19_04.png" width="800">

또는 구글 클라우드 CLI를 설치한 경우(코랩에는 이미 설치되어 있음) 다음 `gsutil` 명령을 사용할 수 있습니다:

In [ ]:
#!gsutil -m cp -r my_mnist_model gs://{bucket_name}/

In [ ]:
from google.cloud import aiplatform

server_image = "gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-8:latest"

aiplatform.init(project=project_id, location=location)
mnist_model = aiplatform.Model.upload(
    display_name="mnist",
    artifact_uri=f"gs://{bucket_name}/my_mnist_model/0001",
    serving_container_image_uri=server_image,
)

**경고**: 이 셀은 버텍스 AI가 컴퓨팅 노드를 프로비저닝할 때까지 기다리므로 실행하는 데 몇 분 정도 걸릴 수 있습니다:

In [ ]:
endpoint = aiplatform.Endpoint.create(display_name="mnist-endpoint")

endpoint.deploy(
    mnist_model,
    min_replica_count=1,
    max_replica_count=5,
    machine_type="n1-standard-4",
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=1
)

In [ ]:
response = endpoint.predict(instances=X_new.tolist())

In [ ]:
import numpy as np

np.round(response.predictions, 2)

In [ ]:
endpoint.undeploy_all()  # 엔드포인트에서 모든 모델 배포 취소
endpoint.delete()

#### GCP에서의 인증 및 권한 부여

1. **OAuth 2.0 인증**
- **사용자 데이터를 대신 접근**하려면 OAuth 2.0 사용.
  - 예: 애플리케이션이 사용자 구글 드라이브 데이터 접근 시 OAuth 인증 필요.
  - 일반적으로 제한된 액세스 권한만 부여하며, 무제한 액세스는 허용되지 않음.
  - 승인은 일정 시간이 지나면 만료되며, 취소 가능.

2. **서비스 계정 (Service Account)**
- 애플리케이션이 **사용자를 대신하지 않고 자체적으로 GCP 서비스**에 액세스할 경우 사용.
- 예: Vertex AI 엔드포인트로 요청을 보내는 웹사이트.
- **서비스 계정 생성 방법**:
  1. GCP 콘솔 → **IAM 및 관리자** → **서비스 계정** → **서비스 계정 만들기** 클릭.
  2. 이름, ID, 설명 입력 후 **계속**.
  3. 필요한 역할(예: Vertex AI 사용자) 부여.
  4. 생성 후 키(JSON 파일)를 다운로드하여 보관.

3. **서비스 계정 인증**
- GCP에서 실행되는 경우:
  - 가상 머신 인스턴스나 앱 엔진과 같은 GCP 리소스에 자동 연결.
- 로컬 환경에서 실행되는 경우:
  - JSON 키 파일을 **`GOOGLE_APPLICATION_CREDENTIALS`** 환경 변수에 설정.
- 쿠버네티스 사용 시:
  - 워크로드 아이덴티티(WIF)를 통해 각 서비스 계정을 매칭하여 안전하게 인증.

4. **JSON 키 파일 관리**
- 키 파일은 서비스 계정의 **민감 정보**를 포함.
- 반드시 안전한 위치에 저장하고, 외부에 노출되지 않도록 관리.

## 버텍스 AI에서 배치 예측 작업 실행하기

In [ ]:
batch_path = Path("my_mnist_batch")
batch_path.mkdir(exist_ok=True)
with open(batch_path / "my_mnist_batch.jsonl", "w") as jsonl_file:
    for image in X_test[:100].tolist():
        jsonl_file.write(json.dumps(image))
        jsonl_file.write("\n")

upload_directory(bucket, batch_path)

In [ ]:
batch_prediction_job = mnist_model.batch_predict(
    job_display_name="my_batch_prediction_job",
    machine_type="n1-standard-4",
    starting_replica_count=1,
    max_replica_count=5,
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=1,
    gcs_source=[f"gs://{bucket_name}/{batch_path.name}/my_mnist_batch.jsonl"],
    gcs_destination_prefix=f"gs://{bucket_name}/my_mnist_predictions/",
    sync=True  # 완료될 때까지 기다리지 않으려면 False로 설정합니다.
)

In [ ]:
batch_prediction_job.output_info  # 추가 코드 - 출력 디렉토리를 표시합니다.

In [ ]:
y_probas = []
for blob in batch_prediction_job.iter_outputs():
    print(blob.name)  # 추가 코드
    if "prediction.results" in blob.name:
        for line in blob.download_as_text().splitlines():
            y_proba = json.loads(line)["prediction"]
            y_probas.append(y_proba)

In [ ]:
y_pred = np.argmax(y_probas, axis=1)
accuracy = np.sum(y_pred == y_test[:100]) / 100

In [ ]:
accuracy

In [ ]:
mnist_model.delete()

GCS에서 만든 모든 디렉토리(즉, 디렉토리 이름의 접두사를 가진 모든 블롭)를 삭제해 보겠습니다:

In [ ]:
for prefix in ["my_mnist_model/", "my_mnist_batch/", "my_mnist_predictions/"]:
    blobs = bucket.list_blobs(prefix=prefix)
    for blob in blobs:
        blob.delete()

#bucket.delete()  # 버킷 자체를 삭제하려면 주석을 제거하고 실행하세요.
batch_prediction_job.delete()

# 모바일 또는 임베디드 디바이스에 모델 배포하기

- **엣지 컴퓨팅 (Edge Computing):**
  - 머신러닝 모델이 중앙 서버 대신 모바일 및 임베디드 디바이스와 같이 데이터 소스에 가까운 곳에서 실행되는 방식을 의미합니다.
  - 장점:
    - 인터넷 연결 없이도 디바이스에서 스마트한 기능 유지.
    - 서버로 데이터 전송이 필요 없어서 지연 시간 감소 및 서버 부하 경감.
    - 데이터가 디바이스에 남아 있어 개인 정보 보호에 유리.

- **단점:**
  - 디바이스의 컴퓨팅 자원이 GPU 서버에 비해 약함.
  - 큰 모델은 디바이스에서 실행이 어려움.
  - RAM과 CPU 사용량이 많아지면 배터리 소모 및 성능 저하.

- **해결 방안:**
  - TFLite(TensorFlow Lite) 라이브러리를 사용하여 모델을 디바이스에 맞게 최적화:
    1. **모델 크기 줄이기**:
       - 다운로드 시간 단축 및 RAM 사용량 감소.
    2. **응답 속도 및 배터리 사용량 최적화**:
       - 계산량을 줄여 예측 속도 개선.
    3. **디바이스 제약 조건에 맞춤화**:
       - 디바이스의 리소스와 제한 사항에 맞게 모델 조정.

---

**TFLite 모델 변환 과정**
- **SavedModel → FlatBuffers 변환**:
  - FlatBuffers는 경량 포맷으로, 모델을 압축하여 메모리 사용량과 로드 시간을 줄임.
  - FlatBuffers의 특징:
    - RAM으로 로드할 때 성능 최적화.
    - 모바일 또는 임베디드 디바이스에서 효율적으로 실행 가능.
  - 결과물은 `.tflite` 형식으로 저장.

---

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(str(model_path))
tflite_model = converter.convert()
with open("my_converted_savedmodel.tflite", "wb") as f:
    f.write(tflite_model)

**모델 최적화와 크기 축소**

1. **연산 최적화**:
   - 예측에 필요하지 않은 연산(예: 훈련 연산)을 제거하거나 가능한 연산을 단순화 및 결합하여 최적화.
   - 예: \(3a + 4x + a + 5x^2\) → \(12x^2 + 4a\).

2. **모델 압축 및 확인 도구**:
   - 사전 훈련된 TFLite 모델(Inception_V1_quant)을 다운로드 후 압축 해제 가능.
   - Netron(https://lutzroeder.github.io/netron)으로 `.pb` 파일이나 `.tflite` 모델의 구조를 그래프로 시각화.

In [ ]:
import requests

# 모델 다운로드 URL
model_url = "https://github.com/google-coral/edgetpu/raw/master/test_data/inception_v1_224_quant.tflite"
output_file = "inception_v1_224_quant.tflite"

# 모델 다운로드
response = requests.get(model_url)
if response.status_code == 200:
    with open(output_file, "wb") as file:
        file.write(response.content)
    print(f"모델 다운로드 완료: {output_file}")
else:
    print(f"모델 다운로드 실패. 상태 코드: {response.status_code}")

In [ ]:
!netron inception_v1_224_quant.tflite

**모델 크기 축소 방법**

1. **반정밀도 숫자 사용**:
   - 32비트 대신 16비트의 반정밀도 실수(half-float)를 사용하여 모델 크기를 절반으로 줄이고, 훈련 속도와 GPU RAM 사용량도 크게 절감.

2. **가중치 양자화**:
   - **Post-training Quantization**:
     - 훈련 후 모델 가중치를 고정 소수점(8비트 정수)로 압축.
     - 32비트 실수와 비교해 4배 더 작은 크기.
   - 대략적인 과정:
     1. 가장 큰 절댓값 \(m\) 찾기.
     2. 값 범위를 \(-m\)에서 \(+m\)까지 고정.
     3. 이 범위를 \(-127\)에서 \(+127\)로 매핑.
   - 예:
     - 원래 가중치 범위: \(-1.5\) ~ \(+1.5\).
     - 매핑 후: \(-127, 0, +127\)로 정수 변환.

<img src="./images/fig_19_05.png" width="800">

3. **`convert()` 메서드 활용**:
   - 변환 최적화 리스트에 `tf.lite.Optimize.DEFAULT`를 추가하여 기본적인 최적화를 수행.

**효과**
- 모델 크기 축소로 다운로드 및 저장 공간 절약.
- 양자화 후 속도 감소는 거의 없으며 RAM 사용량 최소화.
- 모델의 정확도 저하를 최소화하면서도 효율적인 실행 가능.


In [14]:
# 추가 코드 - 케라스 모델을 변환하는 방법을 보여줍니다.
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [15]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [ ]:
tflite_model = converter.convert()
with open("my_converted_keras_model.tflite", "wb") as f:
    f.write(tflite_model)

In [ ]:
!netron my_converted_keras_model.tflite

**양자화(Quantization)의 효과와 활용**
1. **활성화 출력의 양자화**
   - **효과**:
     - CPU 사용량 감소 및 전력 소비 절약.
     - 정수 계산(예: 8비트 정수) 사용으로 속도와 에너지 효율 증가.
   - **활용 사례**:
     - TPU(Edge TPU) 같은 가속 장치는 정수 연산에 최적화되어 있어 효율적으로 처리 가능.

2. **최대 활성화 절댓값 설정**
   - 양자화를 수행하기 전 **활성화 값 통계**를 수집해 최적의 절댓값 범위를 설정.
   - **Calibration 단계**: 대표 샘플 데이터를 모델에 입력해 활성화 범위를 결정.
   - **Tip**: 적은 샘플 데이터로도 활성화 통계를 측정하여 모델을 통과시키는 양자화 작업 가능.

3. **정확도와 손실 문제**
   - 양자화는 **정확도를 약간 희생**하는 대가로 전력과 속도를 대폭 향상.
   - **정확도 손실을 줄이는 방법**:
     - "양자화를 고려한 훈련(Quantization-aware training)" 사용.
     - 양자화된 환경을 고려해 추가 연산을 포함하여 훈련 진행.
   - 최종 가중치가 양자화된 값에 안정적으로 수렴.

# 웹 페이지에서 모델 실행하기

---

**1. 브라우저에서 모델 실행의 장점**
- **느린 네트워크 상황에서 유용**:
  - 사용자 연결이 제한적이거나 느린 상황에서, 클라이언트 측에서 모델을 실행하면 앱이 안정적으로 작동 가능.
  - 예: 등산객을 위한 오프라인 웹사이트.
  
- **응답 속도 향상**:
  - 예측 시 서버에 쿼리하지 않아도 되므로 웹사이트의 응답 속도가 크게 개선. (온라인 게임 등)

- **데이터 보안**:
  - 비공개 데이터를 클라이언트 측에서 처리하여, 개인 정보가 외부로 유출되지 않도록 보호.

---

**2. TensorFlow.js(TFJS)를 활용한 클라이언트 측 모델 실행**
- **TensorFlow.js 소개**:
  - 자바스크립트 라이브러리로, TFLite 모델을 로드하고 사용자 브라우저에서 실행 가능.
  - 사전 훈련된 모델(MobileNet 등)을 로드하여 이미지 분류, 예측 등을 수행.

- **실습 사이트**:
  - [Glitch](https://homl.info/tfjswpa): 자바스크립트 코드 예제를 테스트할 수 있는 사이트.
  - 사이트 내 `[PREVIEW]` 버튼 클릭으로 코드 실행 결과 확인 가능.

```html

import "https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@latest";
import "https://cdn.jsdelivr.net/npm/@tensorflow-models/mobilenet@1.0.0";

const image = document.getElementById("image");

mobilenet.load().then(model => {
    model.classify(image).then(predictions => {
        for (var i = 0; i < predictions.length; i++) {
            let className = predictions[i].className;
            let proba = (predictions[i].probability * 100).toFixed(1);
            console.log(className + ": " + proba + "%");
        }
    });
});


```
- 다음 명령어로 webserver 실행 필요.

```bash
python -m http.server 8000
```

---

**3. 프로그레시브 웹 앱(PWA)으로의 확장**
- **PWA란?**:
  - 클라이언트 측에서 작동하며 독립 실행형 앱처럼 설치 및 실행 가능.
  - 심지어 모바일 디바이스에서 독립 앱처럼 작동.

- **PWA의 특징**:
  - 홈 화면에 추가하여 독립적인 앱처럼 실행 가능.
  - 오프라인에서도 작동 가능(서비스 워커 기능 사용).
  - 백그라운드 작업 수행 및 푸시 메시지 지원.

- **PWA 코드 샘플**:
  - Glitch의 [PWA 코드 예제](https://homl.info/wpacode)에서 확인 가능.

- **활용 예시**
  - TFJS를 활용한 웹 애플리케이션 제작으로 이미지 분류, 예측, 데이터 분석 등 다양한 작업 가능.
  - PWA로 확장하여 모바일 및 웹에서 일관된 사용자 경험 제공.

---

**4. TFJS와 GPU 활용**
- **TFJS의 GPU 지원**:
  - TFJS는 **WebGL**을 사용하여 GPU 가속을 지원.
  - NVIDIA GPU뿐만 아니라 최신 웹 브라우저에서 광범위한 GPU 카드 지원.

- **사용 사례**:
  - 브라우저에서 모델 학습 가능.
  - 클라이언트 측에서 사용자 데이터를 비공개로 유지하며 학습(연합 학습, Federated Learning).
  - 모델 학습 후 로컬 데이터 기반으로 미세 튜닝 가능.

- **관련 자료**:
  - [연합 학습](https://tensorflow.org/federated) 참조.

이 섹션의 코드 예제는 웹 앱을 무료로 만들 수 있는 웹사이트인 glitch.com에서 호스팅됩니다.

* https://homl.info/tfjscode: 사전 학습된 모델을 로드하고 이미지를 분류하는 간단한 TFJS 웹 앱입니다.
* https://homl.info/tfjswpa: WPA로 설정된 동일한 웹 앱. 모바일 장치를 포함한 다양한 플랫폼에서 이 링크를 열어 보세요.
** https://homl.info/wpacode: 이 WPA의 소스 코드입니다.
* https://tensorflow.org/js: TFJS 라이브러리.
** https://www.tensorflow.org/js/demos: 재미있는 데모 몇 가지를 소개합니다.

# 계산 속도를 높이기 위해 GPU 사용하기

- **배경:** 
  - 대규모 신경망을 CPU만으로 훈련 시 시간이 매우 오래 걸림.
  - GPU 사용 시 훈련 시간을 크게 단축할 수 있음.

- **GPU 사용의 이점:**
  - 훈련 시간을 몇 시간 또는 몇 분으로 줄일 수 있음.
  - 모델 실험 및 데이터 업데이트 주기를 더 빠르게 실행 가능.

- **GPU 활용 방법:**
  - 구글 코랩에서 GPU 런타임 변경으로 간단히 사용 가능.
  - 코드 호환성 유지: GPU가 없는 환경에서도 동일하게 실행 가능.
  - 여러 GPU가 포함된 노드에서도 작동.

- **구체적인 활용:**
  - Vertex AI 모델 배포 시 GPU 선택 가능 (`endpoint.deploy()`).
  - 단일 머신의 CPU 및 GPU, 다수 GPU 장치 활용 방법 논의 예정.
  - 서버 계산 분산 방법도 추가로 설명.

<img src="./images/fig_19_06.png" width="800">

## GPU 구매하기

- **구매 필요성 판단:**
  - GPU를 장기적으로 많이 사용할 경우 직접 구매하는 것이 경제적으로 유리.
  - 로컬 환경에서 데이터 처리 및 훈련을 원하는 경우 적합.

- **GPU 구매 시 고려 사항:**
  - 작업 특성에 따라 필요한 RAM 용량, 대역폭, 코어 수, 냉각 시스템 등 검토 필요.
  - 예시: 이미지 처리 및 NLP는 최소 10GB RAM 권장.
  - Tim Dettmery의 블로그(https://homl.info/66)에서 상세 정보 확인 가능.

- **호환성 및 지원:**
  - TensorFlow와 CUDA 계산 능력 3.5 이상의 Nvidia GPU만 지원.
  - 다른 제조사의 장치를 지원하는 가능성도 있으므로 TensorFlow 문서 참고.

- **Nvidia GPU 설정:**
  - Nvidia 드라이버 및 CUDA (Compute Unified Device Architecture) 라이브러리 설치 필요.
    - NVIDIA가 만든 **GPU를 위한 소프트웨어 개발 도구**
  - cuDNN (CUDA Deep Neural Network Library)을 통해 DNN 연산 가속 가능 (추론 및 학습 속도 향상).
    - CUDA 위에서 동작하는 **딥러닝 최적화 라이브러리**
  - Nvidia 딥러닝 SDK 및 개발자 계정 필요.

<img src="./images/fig_19_07.png" width="800">

- **설치 확인:**
  - Nvidia 드라이버 및 라이브러리 설치 후 `nvidia-smi` 명령어를 사용하여 GPU 설치 상태 점검 가능.
  - 이 명령어는 사용 가능한 GPU 카드, 실행 중인 프로세스, 카드 사양 등을 확인할 수 있음.

In [ ]:
!nvidia-smi

텐서플로우가 GPU를 볼 수 있는지 확인해 보겠습니다:

In [ ]:
physical_gpus = tf.config.list_physical_devices("GPU")
physical_gpus

텐서플로 스크립트에서 GPU \#0과 \#1(PCI 순서 기준)만 사용하려면 스크립트를 시작하기 전에 환경 변수 `CUDA_DEVICE_ORDER=PCI_BUS_ID`와 `CUDA_VISIBLE_DEVICES=0,1`을 설정합니다. 또는 스크립트 자체에서 텐서플로를 사용하기 전에 지정할 수 있습니다.

## GPU RAM 관리하기

- **기본 동작:**
  - TensorFlow는 실행 시 자동으로 사용 가능한 GPU의 대부분 RAM을 할당함.
  - 다른 프로그램이 GPU를 사용할 경우 RAM 부족 현상이 발생할 수 있음.

- **다중 프로그램 실행 시 GPU RAM 분배:**
  - 동일 컴퓨터에서 여러 TensorFlow 프로그램을 실행하려면 GPU RAM을 균등하게 나누어 사용해야 함.
  - `CUDA_VISIBLE_DEVICES` 환경 변수를 설정해 특정 GPU를 각 프로세스에 할당 가능.
  - `CUDA_DEVICE_ORDER` 환경 변수를 `PCI_BUS_ID`로 설정하여 GPU ID를 고정적으로 유지 가능.

- **예시 명령어:**
  - GPU 카드 4개 중 2개를 각기 다른 프로그램에 할당하는 경우:

    ```bash
    CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=0,1 python3 program_1.py
    CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=2,3 python3 program_2.py
    ```

- **환경 변수 설정:**
  - TensorFlow 실행 전에 아래와 같이 Python 스크립트에서 환경 변수를 설정 가능:
  
    ```python
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
    ```

<img src="./images/fig_19_08.png" width="800">

- **GPU RAM 할당량 제한**
  - TensorFlow가 GPU마다 특정 양의 RAM만 사용하도록 설정 가능.
  - 예시 코드: 각 GPU에 2GiB 메모리 할당
    ```python
    for gpu in physical_gpus:
        tf.config.set_logical_device_configuration(
            gpu,
            [tf.config.LogicalDeviceConfiguration(memory_limit=2048)]
        )
    ```
  - 이를 통해 4GiB RAM을 가진 GPU 4개를 두 개의 프로그램에서 나누어 사용할 수 있음.

  <img src="./images/fig_19_09.png" width="800">

In [ ]:
#for gpu in physical_gpus:
#    tf.config.set_logical_device_configuration(
#        gpu,
#        [tf.config.LogicalDeviceConfiguration(memory_limit=2048)]
#    )

- **필요할 때만 메모리 점유**
  - 점진적으로 텐서플로가 메모리를 점유하도록 하려면(프로세스가 종료될 때만 메모리를 해제합니다):
  - 예시 코드:
    ```python
    for gpu in physical_gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    ```

In [ ]:
#for gpu in physical_gpus:
#    tf.config.experimental.set_memory_growth(gpu, True)

- `TF_FORCE_GPU_ALLOW_GROWTH`를 `True`로 설정하면 TensorFlow가 필요에 따라 GPU 메모리를 점유 및 해제 가능.
  - 이 방법은 메모리 고정으로 인한 충돌을 방지하지만, 결정적인 행동을 보장하기 어려움.
  - 주로 코랩 런타임이나 단일 GPU를 다중 논리적 장치로 나누는 경우 유용.
  - 예를 들어, GPU #0을 두 개의 논리적 GPU로 나누고 각 논리적 장치에 2GiB의 메모리 한도를 설정:
    ```python
    tf.config.set_logical_device_configuration(
        physical_gpus[0],
        [
            tf.config.LogicalDeviceConfiguration(memory_limit=2048),
            tf.config.LogicalDeviceConfiguration(memory_limit=2048)
        ]
    )
    ```

- 물리적 GPU를 두 개의 논리적 GPU로 분할합니다:
```python
tf.config.set_logical_device_configuration(
   physical_gpus[0],
   [tf.config.LogicalDeviceConfiguration(memory_limit=2048),
    tf.config.LogicalDeviceConfiguration(memory_limit=2048)]
)
```

In [ ]:
# tf.config.set_logical_device_configuration(
#    physical_gpus[0],
#    [tf.config.LogicalDeviceConfiguration(memory_limit=2048),
#     tf.config.LogicalDeviceConfiguration(memory_limit=2048)]
# )

- 생성된 논리적 GPU를 확인하는 코드:
    ```python
    logical_gpus = tf.config.list_logical_devices("GPU")
    print(logical_gpus)
    ```
- 결과: `/gpu:0`, `/gpu:1`와 같은 논리적 장치 이름 확인 가능.


In [9]:
# logical_gpus = tf.config.list_logical_devices("GPU")
# logical_gpus

## 디바이스에 연산 및 변수 배치하기

- **기본 동작:**
  - TensorFlow는 `tf.data`를 사용해 연산과 변수를 자동으로 적절히 배치.
  - 세부적인 제어가 필요한 경우, 디바이스별로 연산과 변수를 수동 배치 가능.

- **배치 원칙:**
  1. 데이터 전처리는 CPU에서 처리, 신경망 연산은 GPU에서 수행.
  2. GPU는 대역폭 제약이 있으므로 불필요한 데이터 전송을 최소화해야 함.
  3. CPU RAM은 여유롭게 사용 가능하지만, GPU RAM은 제한적이므로 중요 변수만 배치 권장.

- **기본 배치 규칙:**
  - GPU 커널이 있는 변수와 연산은 첫 번째 GPU (`/gpu:0`)에 할당.
  - GPU 커널이 없는 경우 CPU (`/cpu:0`)에 할당.

- **변수 및 연산 배치 확인:**
  - TensorFlow 변수의 `device` 속성을 통해 변수나 연산이 할당된 디바이스 확인 가능.
    ```python
    a = tf.Variable([1., 2., 3.])  # float32 변수는 GPU로 이동
    print(a.device)  # '/job:localhost/replica:0/task:0/device:GPU:0'

    b = tf.Variable([1, 2, 3])  # int32 변수는 CPU로 이동
    print(b.device)  # '/job:localhost/replica:0/task:0/device:CPU:0'
    ```

- **특징:**
  - 정수 변수 등 GPU 커널이 없는 연산은 기본적으로 CPU에 배치됨.
  - `/job:localhost/replica:0/task:0`는 무시 가능하며 디바이스의 실제 위치를 나타냄.

모든 변수 및 연산 배치를 기록하려면(이 작업은 텐서플로를 임포팅한 직후에 실행해야 합니다):

In [ ]:
#tf.get_logger().setLevel("DEBUG")  # 디폴트 로그 수준은 INFO입니다.
#tf.debugging.set_log_device_placement(True)

In [ ]:
a = tf.Variable([1., 2., 3.])  # float32 변수는 GPU로 이동합니다.
a.device

In [ ]:
b = tf.Variable([1, 2, 3])  # int32 변수는 CPU로 이동합니다.
b.device

`tf.device()` 컨텍스트를 사용하여 원하는 장치에 변수 및 연산을 수동으로 배치할 수 있습니다:

In [ ]:
with tf.device("/cpu:0"):
    c = tf.Variable([1., 2., 3.])

c.device

존재하지 않거나 커널이 없는 장치를 지정하면 텐서플로는 자동으로 기본 장치를 사용합니다:

In [ ]:
# 추가 코드

with tf.device("/gpu:1234"):
    d = tf.Variable([1., 2., 3.])

d.device

텐서플로에서 존재하지 않는 장치를 사용하려고 할 때 기본 장치로 되돌아가지 않고 예외를 발생시키려면:

In [ ]:
tf.config.set_soft_device_placement(False)

# 추가 코드
try:
    with tf.device("/gpu:1000"):
        d = tf.Variable([1., 2., 3.])
except tf.errors.InvalidArgumentError as ex:
    print(ex)

tf.config.set_soft_device_placement(True)  # 추가 코드 - 소프트 배치로 돌아가기

## 다중 장치에서 병렬 실행하기

텐서플로(TensorFlow)는 병렬 처리가 가능하도록 설계되어 있어, 여러 장치를 활용해 연산을 분산시켜 효율적으로 실행할 수 있습니다.

1. **작업 분석 및 평가**
   - 텐서플로는 실행 시 그래프를 분석해 각 연산의 의존성을 계산합니다.
   - 의존성이 없는 연산(소스 연산)은 평가 큐(evaluation queue)에 추가됩니다.
   - 연산이 평가되면 해당 연산에 의존하는 다른 연산들의 의존성 카운터가 감소하여 실행 준비 상태가 됩니다.

2. **CPU 병렬 처리**
   - CPU는 `inter-op` 스레드 풀과 `intra-op` 스레드 풀을 사용합니다.
     - **Inter-op 스레드 풀**: 여러 연산이 다른 CPU 코어에서 병렬로 실행됩니다.
     - **Intra-op 스레드 풀**: 하나의 연산 내에서 병렬 처리를 수행합니다.
   - 예를 들어, 다중 스레드 CPU 커널이 있는 연산은 `intra-op` 스레드 풀을 통해 효율적으로 병렬 처리됩니다.

3. **GPU 병렬 처리**
   - GPU는 텐서플로에서 CUDA 및 cuDNN 라이브러리를 사용해 병렬 처리를 수행합니다.
   - GPU 자체적으로 스레드 풀을 가지고 있어 `inter-op` 스레드 풀이 필요 없습니다.
   - GPU의 연산은 대부분 GPU 스레드를 사용해 병렬로 실행됩니다.

4. **실행 과정 예시**
   - [그림 19-10]을 참고로, 연산 A, B, C는 의존성이 없으므로 바로 평가됩니다.
   - CPU와 GPU의 큐로 각각 분배된 후 병렬로 계산됩니다.
   - D, E는 C의 완료를 기다린 후 평가됩니다.
   - 의존성이 모두 해소된 F가 마지막으로 실행됩니다.

5. **순차 실행 보장**
   - 변수나 리소스를 수정하는 연산의 경우, 텐서플로는 코드 작성 순서를 보장해 순차적으로 실행합니다.


<img src="./images/fig_19_10.png" width="800">


6. **여러 GPU에서 모델 병렬 훈련**
   - 각 GPU에 하나의 모델을 배치하여 병렬로 훈련 가능.
   - CUDA_DEVICE_ORDER와 CUDA_VISIBLE_DEVICES를 사용하여 특정 GPU를 지정.
   - 하이퍼파라미터 튜닝에 유용.
   - 예: GPU 2개로 병렬 훈련 시, 단일 GPU로 하나의 모델을 학습 시키는데 1시간 걸릴 작업이,  2개 GPU로도 동일한 시간 내 두개의 모델을 병렬 수행 가능.

7. **GPU와 CPU의 협업**
   - GPU가 모델을 훈련하는 동안 CPU는 데이터를 전처리.
   - `prefetch()` 메서드를 활용하여 GPU가 데이터를 요청하기 전에 미리 준비 가능.

8. **CNN과 병렬 처리**
   - 입력 데이터를 2개로 나누고 각 데이터를 다른 GPU에서 병렬 처리.
   - 결과를 합쳐 더욱 빠르게 CNN 계산 가능.

9. **효율적인 앙상블 모델 구축**
   - GPU에서 개별 모델을 병렬 훈련 후, 앙상블 결과를 최종적으로 통합하여 예측 성능 향상.


inter-op 또는 intra-op 스레드 수를 설정하려는 경우(CPU 포화를 방지하거나 완벽하게 재현 가능한 테스트 케이스를 실행하기 위해 텐서플로를 단일 스레드로 만들고자 하는 경우 유용할 수 있습니다):

In [ ]:
#tf.config.threading.set_inter_op_parallelism_threads(10)
#tf.config.threading.set_intra_op_parallelism_threads(10)

# 다중 장치에서 모델 훈련하기

다중 장치에서 모델을 훈련하는 두 가지 방법:

1. **모델 병렬화 (Model Parallelism)**  
   - 하나의 모델을 여러 장치에 나누어 분할하여 각 장치가 모델의 일부를 처리.

2. **데이터 병렬화 (Data Parallelism)**  
   - 동일한 모델 복사본을 여러 장치에 배치하고, 데이터를 나누어 각 장치에서 병렬로 처리.

### 모델 병렬화

- **개념**  
  - 모델의 각 부분을 여러 장치에 나누어 병렬적으로 실행하는 방식.

- **장점**  
  - 특정 구조의 신경망(예: 합성곱 신경망)의 경우 효율적으로 병렬화 가능.

<img src="./images/fig_19_11.png" width="800">

- **단점**  
  - **통신 지연**: 장치 간 통신이 빈번하며 특히 다른 머신 사이에서는 속도가 느려짐.  
  - **의존성 문제**: 각 층이 이전 층의 출력을 기다려야 하므로 병렬 효율성이 저하됨.  
  - **구현 복잡성**: 많은 셀을 병렬로 실행해야 하는 경우 설정과 조율이 복잡.

<img src="./images/fig_19_12.png" width="800">

- **적용 사례**  
  - 순환 신경망(LSTM): 실질적으로 병렬화보다는 한 GPU에서 처리하는 것이 더 효율적.
  - 합성곱 신경망: 일부 층이 부분적으로만 연결되어 있어 병렬화 효율이 비교적 높음.

<img src="./images/fig_19_13.png" width="800">

## 데이터 병렬화

- **개념**  
  - 동일한 모델 복사본을 여러 장치에 배치하고, 각 장치에 다른 미니배치를 할당하여 병렬적으로 훈련.  
  - 각 장치에서 계산된 그레디언트를 평균화하여 모델 파라미터를 업데이트.  
  - **SPMD(Single Program, Multiple Data)** 접근 방식.

- **장점**  
  - **효율성:** 병렬화 구현이 비교적 간단하며, 통신 비용이 모델 병렬화보다 적음.  
  - **확장성:** GPU나 장치를 추가할수록 병렬 처리 능력 향상.  

- **단점**  
  - 결과를 통합하고 평균화하는 단계에서 통신 오버헤드 발생 가능.  
  - 데이터의 불균형이나 동기화 문제가 있을 경우 성능 저하 가능.


### 미러드 전략을 사용한 데이터 병렬화

- **개념**  
  - 모든 GPU에 동일한 모델 파라미터 복사본을 생성하고 항상 동일한 상태를 유지하며 훈련.  
  - 이를 **미러드 전략(Mirrored Strategy)** 이라고 부름.

<img src="./images/fig_19_14.png" width="800">

- **장점**  
  - **효율성:** 단일 머신에서 사용할 때 매우 효과적.  
  - **동일 상태 유지:** 모든 복제된 모델이 항상 동일한 파라미터를 유지.

- **알고리즘**  
  - **올리듀스(AllReduce):**  
    - 모든 GPU에서 계산된 그레디언트의 평균을 효율적으로 계산하고 이를 모든 GPU에 배포.  
    - 여러 노드가 협력하여 평균, 합, 최댓값 등을 계산하는 방식.  
    - 리듀스(reduce) 연산을 효율적으로 수행.
      - 리듀스(Reduce) 연산은 분산 컴퓨팅에서 여러 데이터를 하나의 값으로 집계하는 연산을 의미

- **한계**  
  - 올리듀스는 효율적이지만 노드 간 통신 비용이 발생할 수 있음.  


### 중앙 집중적인 파라미터를 사용한 데이터 병렬화

- **개념**  
  - 계산을 수행하는 GPU 장치(워커) 외에 **파라미터 서버**(Parameter Server)를 사용하여 모델 파라미터를 저장 및 업데이트.  
  - 미러드 전략과 달리 중앙 집중적으로 파라미터를 관리하며, 동기 업데이트와 비동기 업데이트 방식 사용 가능.


**동기 업데이트 (Synchronous Update)**  
- **작동 방식**  
  - 모든 워커에서 그레디언트를 계산한 후 이를 평균화하여 모델 파라미터를 업데이트.  
  - 업데이트 완료 후 모델 파라미터를 모든 장치에 복사.

<img src="./images/fig_19_15.png" width="800">

- **장점** : 모든 워커가 동기화된 상태로 동일한 모델 파라미터를 사용 가능.

- **단점** : 느린 워커가 다른 워커들의 처리를 지연시키는 병목현상 발생.  

- **최적화 방법 (Tip)**  
  - 느린 복제 모델(10% 정도)을 무시하고 빠른 모델들로만 평균 그레디언트를 계산.  
  - 예: 복제 모델 20개 중 가장 느린 2개를 제외하고 나머지 18개로 작업 진행.  

 **비동기 업데이트 (Asynchronous Update) 요약**

- **개념**  
  - 복제 모델이 각자 독립적으로 그레디언트를 계산한 후 바로 모델 파라미터를 업데이트.  
  - 동기화 과정이 없으며, 각 복제 모델이 다른 복제 모델을 기다리지 않음.  

**장점**  
1. **효율성** : 병렬 처리의 장점이 극대화되어 더 많은 훈련 스텝을 빠르게 실행 가능.  
2. **대역폭 감소** : 동기화 지연이 없어 통신 비용 감소.  

**단점**  
1. **불안정성:**  
   - 각 복제 모델이 독립적으로 작동하여 모델 상태가 불안정해질 위험.  
   - 복제 모델 간의 업데이트 간격으로 인해 **낡은 그레디언트(stale gradient)** 문제가 발생.  
   - 낡은 그레디언트는 훈련의 수렴 속도를 느리게 하고 학습 곡선의 불안정을 초래.

<img src="./images/fig_19_16.png" width="800">

2. **균형 문제:**  
   - 특정 복제 모델이 파라미터를 자주 업데이트하여 다른 복제 모델과 방향성이 달라질 위험.

**해결 방법**  
1. **학습률 감소:**  
   - 지나친 업데이트로 인한 불안정성을 완화.  
2. **낡은 그레디언트 무시:**  
   - 너무 오래된 그레디언트는 버림.  
3. **미니배치 크기 조정:**  
   - 데이터 샘플 크기를 조절해 균형을 맞춤.  
4. **준비 단계 도입:**  
   - 하나의 복제 모델만 사용하여 처음 몇 번의 에포크 시작, 초기 에포크 동안 안정적으로 작동하도록 설정.  

### 대역폭 포화

- **문제 정의**  
  동기 업데이트나 비동기 업데이트 중 어떤 것을 사용하든, 훈련 시 모델의 파라미터와 그레이디언트를 중앙 서버와 GPU 간에 전송해야 하는 문제가 발생.  
  특히, 병렬적으로 작업하는 경우 각 GPU에서 계산된 그레이디언트를 공유해야 하므로 대역폭 포화 문제가 생김.

- **성능 한계**  
  GPU를 추가하더라도 네트워크 전송 속도 제한으로 성능 향상이 어렵고, 대규모 딥러닝 모델에서는 이런 문제로 인해 훈련 속도가 느려짐.

- **대규모 모델에서의 영향**  
  - 그레이디언트의 양이 많아질수록 대역폭 문제가 더 심각해짐.
  - 희소 모델의 규모를 늘리기가 더 쉽다는 구체적인 사례:
    - **신경망 기계 번역:** GPU 8개에서 6배 속도 증가.
    - **인셉션/이미지넷:** GPU 50개에서 32배 속도 증가.
    - **랭크브레인:** GPU 500개에서 300배 속도 증가.

**해결 방법: 병렬화 기법**
- **PipeDream 시스템**  
  모델 병렬화와 데이터 병렬화를 결합한 새로운 방식 제안.  
  - **스테이지(Stage) 분할:** 모델을 여러 부분으로 나눠 각기 다른 머신에서 훈련.  
  - **비동기 처리:** 데이터와 그레이디언트를 병렬로 처리하면서 대역폭 문제 완화.
    - **작동 방식**  
      - 입력 데이터를 작은 미니배치로 나눠 각각 처리.  
      - 역전파 단계에서 그레이디언트를 순차적으로 업데이트.  
      - 각 단계가 독립적으로 작동하며, 대역폭 부담을 줄이기 위해 "미리보기 전략" 사용 가능.

    <img src="./images/fig_19_17.png" width="800">
    
    - **기대 효과**  
      - PipeDream과 같은 새로운 병렬화 기술은 GPU 자원을 효율적으로 활용하면서 대규모 모델 훈련을 가속화 가능.
     
    - **PipeDream의 문제점**  
      - 스테이지를 거치며 역전파 시 정확도가 떨어지고 발산 가능성 증가.  
      - 특정 스테이지에서 계산된 그레이디언트가 다른 스테이지에서 제대로 반영되지 않는 문제.  
      - 이러한 문제를 해결하기 위해 **가중치 스태싱(weight stashing)** 기법을 제안:
        - 정방향 계산 중에 가중치를 저장하여 역전파 단계에서도 일관되게 사용.
        - 이를 통해 정확도 문제를 해결하고 확장성 개선.
   
- **Pathways 시스템**  
  구글 연구팀이 개발한 시스템으로, TPU 활용률을 최대화하고 대규모 모델을 효율적으로 훈련하기 위한 새로운 접근법.  
  - **특징:** 
    - 자동화된 모델 병렬 처리.
    - 비동기 갱 스케줄링(gang scheduling)을 통해 대규모 TPU 클러스터에서 작업 병렬화를 극대화.
    - TPU 6,000개 이상에서 대규모 언어 모델 훈련에 사용.
    - 하드웨어 활용률이 100%에 가까움.

    - Pathways는 아직 공개되지 않았으나, 향후 베타스에서도 유사한 시스템이 대규모 모델 훈련에 사용될 가능성이 높음.  
- **대역폭 문제 해결을 위한 방법:**  
    - 저렴한 GPU를 여러 대 사용하는 대신, 강력한 GPU 소수를 활용.
    - 네트워크 부하를 줄이기 위해 여러 GPU를 하나의 서버에 집중 연결.
    - 데이터 전송 크기를 줄이기 위해 32비트(tf.float32)를 16비트(tf.float16)로 변환.
    - **중앙집중식 파라미터 서버 활용**  
        - 파라미터 서버 여러 대를 추가하여 각 서버에 네트워크 부하 분산.  
        - 이를 통해 대역폭 포화 문제를 최소화하고 훈련 속도를 높일 수 있음.

## 분산 전략 API를 사용한 대규모 훈련

In [ ]:
# 추가 코드 - 케라스를 사용하여 MNIST용 CNN 모델을 생성합니다.
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Reshape([28, 28, 1], input_shape=[28, 28],
                                dtype=tf.uint8),
        tf.keras.layers.Rescaling(scale=1 / 255),
        tf.keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
                               padding="same"),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                               padding="same"),
        tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                               padding="same"),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=64, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=10, activation="softmax"),
    ])

**1. 개요**
- 텐서플로는 여러 장치와 머신에서 모델을 분산 처리하는 복잡성을 대신 처리해주는 **분산 전략 API**를 제공.
- **MirroredStrategy**:
  - 모든 GPU에서 데이터 병렬화를 사용해 동기화.
  - **scope()** 메서드로 분산 컨텍스트를 설정한 뒤 모델 생성 및 컴파일 과정을 실행.
  - 이후 **fit()** 메서드로 훈련.

In [ ]:
# 랜덤 시드를 42로 설정하여 재현성 확보
tf.random.set_seed(42)

# MirroredStrategy 객체 생성 - 모든 GPU에서 동기화된 데이터 병렬화 수행
strategy = tf.distribute.MirroredStrategy()

# strategy.scope() 컨텍스트에서 모델 생성 및 컴파일
with strategy.scope():
    # 일반적인 케라스 모델 생성 
    model = create_model()
    
    # 모델 컴파일 - 손실 함수, 옵티마이저, 평가 지표 설정
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2), 
                  metrics=["accuracy"])

# 배치 크기 설정 - GPU 복제본 수로 나누어지도록 설정
batch_size = 100

# 모델 훈련 - 10 에포크 동안 훈련 및 검증 데이터로 평가
model.fit(X_train, y_train, epochs=10,
          validation_data=(X_valid, y_valid), batch_size=batch_size)

- `batch_size`는 GPU 개수로 나누어져 각 복제 모델에 할당.
- 모든 복제 모델에 동일한 크기의 배치가 전달되어 병렬 처리 속도가 증가.
- `model.weights`는 `MirroredVariable` 타입으로 GPU 간 동기화를 지원.

In [ ]:
type(model.weights[0])

- **predict()**: 모든 복제 모델에 배치를 나눠 병렬로 예측을 수행.


In [ ]:
model.predict(X_new).round(2)  # 추가 코드 - 배치가 모든 복제본에 분할됩니다.

- **save()**: 일반적인 형태로 모델을 저장. 특정 GPU가 아닌 모든 장치에서 로드 가능.

In [ ]:
# 추가 코드 - 모델을 저장해도 분산 전략이 보존되지 않음을 보여줍니다.
model.save("my_mirrored_model", save_format="tf")
model = tf.keras.models.load_model("my_mirrored_model")
type(model.weights[0])

In [ ]:
with strategy.scope():
    model = tf.keras.models.load_model("my_mirrored_model")

In [ ]:
type(model.weights[0])

사용할 GPU 리스트를 지정하려는 경우:

**MirroredStrategy 옵션 변경**
- **기본 설정**:
  - NCCL(NVIDIA Collective Communications Library)을 사용해 GPU 간 평균 계산.
- **다른 옵션**:
  - **tf.distribute.HierarchicalCopyAllReduce**:
    - NCCL 대신 GPU와 모델 구조에 따라 더 적합한 옵션.
  - **tf.distribute.ReductionToOneDevice**:
    - 모든 데이터를 한 장치로모아 처리.


In [ ]:
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

In [ ]:
strategy = tf.distribute.MirroredStrategy(
    cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

**CentralStorageStrategy**
- **MirroredStrategy**의 대안
- 중앙 집중식 파라미터 서버에서 데이터 병렬화를 수행.

In [ ]:
strategy = tf.distribute.experimental.CentralStorageStrategy()

In [ ]:
# Google Colab에서 TPU로 훈련하기:
#if "google.colab" in sys.modules and "COLAB_TPU_ADDR" in os.environ:
#  tpu_address = "grpc://" + os.environ["COLAB_TPU_ADDR"]
#else:
#  tpu_address = ""
#resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address)
#tf.config.experimental_connect_to_cluster(resolver)
#tf.tpu.experimental.initialize_tpu_system(resolver)
#strategy = tf.distribute.experimental.TPUStrategy(resolver)

## 텐서플로 클러스터에서 모델 훈련하기

**텐서플로 클러스터 구성 요소**

텐서플로 클러스터는 일반적으로 서로 다른 컴퓨터에서 병렬로 실행되며 신경망 훈련이나 실행과 같은 작업을 완료하기 위해 서로 대화하는 텐서플로 프로세스의 그룹입니다. 클러스터의 각 TF 프로세스를 "태스크"(또는 "TF 서버")라고 합니다. IP 주소, 포트, 타입(역할 또는 작업이라고도 함)이 있습니다. 타입은 `"worker"`, `"chief"`, `"ps"`(파라미터 서버) 또는 `"evaluator"`가 될 수 있습니다:
* **워커**는 일반적으로 하나 이상의 GPU가 있는 컴퓨터에서 계산을 수행합니다.
* **치프**는 계산도 수행하지만, 텐서보드 로그 작성이나 체크포인트 저장과 같은 추가 작업도 처리합니다. 클러스터에는 하나의 치프가 있습니다. 정의되지 않은 경우 워커 #0이 치프가 됩니다.
* **파라미터 서버**(ps)는 변수 값만 추적하며, 일반적으로 CPU 전용 머신에 있습니다.
* **이밸류에이터**는 당연히 평가를 담당합니다. 일반적으로 클러스터에는 하나의 이밸류에이터가 있습니다.

같은 유형의 태스크 집합을 흔히 "작업(job)"이라고 합니다. 예를 들어 "워커" 작업은 모든 워커 태스크의 집합입니다.

텐서플로 클러스터를 시작하려면 먼저 클러스터를 정의해야 합니다. 즉, 모든 작업(IP 주소, TCP 포트 및 타입)을 지정해야 합니다. 예를 들어, 다음 클러스터 사양은 3개의 태스크(워커 2개, 파라미터 서버 1개)가 있는 클러스터를 정의합니다. 작업당 하나의 키가 있는 사전이며, 값은 태스크 주소 목록입니다:

<img src="./images/fig_19_18.png" width="800">

In [ ]:
cluster_spec = {
    "worker": [
        "machine-a.example.com:2222",     # /job:worker/task:0
        "machine-b.example.com:2222"      # /job:worker/task:1
    ],
    "ps": ["machine-a.example.com:2221"]  # /job:ps/task:0
}

클러스터의 모든 태스크는 서버의 다른 모든 태스크와 통신할 수 있으므로 방화벽이 해당 포트에서 컴퓨터 간의 모든 통신을 승인하도록 구성해야 합니다(일반적으로 모든 컴퓨터에서 동일한 포트를 사용하는 것이 간단합니다).

태스크가 시작되면 어떤 태스크인지 타입과 인덱스(태스크 인덱스는 태스크 ID라고도 함)를 알려주어야 합니다. 클러스터 사양과 현재 태스크의 타입 및 아이디를 모두 한 번에 지정하는 일반적인 방법은 프로그램을 시작하기 전에 `TF_CONFIG` 환경 변수를 설정하는 것입니다. 클러스터 사양(``cluster`` 키 아래)과 시작할 태스크의 타입 및 인덱스(``task`` 키 아래)가 포함된 JSON 인코딩된 딕셔너리여야 합니다. 예를 들어, 다음 `TF_CONFIG` 환경 변수는 위와 동일한 클러스터(워커 2개, 파라미터 서버 1개)를 정의하고, 시작할 태스크를 워커 \#0으로 지정합니다:

In [ ]:
os.environ["TF_CONFIG"] = json.dumps({
    "cluster": cluster_spec,
    "task": {"type": "worker", "index": 0}
})

일부 플랫폼(예: 구글 버텍스 AI)에서는 이 환경 변수를 자동으로 설정합니다.

텐서플로의 `TFConfigClusterResolver` 클래스는 이 환경 변수에서 클러스터 구성을 읽습니다:

In [ ]:
resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()
resolver.cluster_spec()

In [ ]:
resolver.task_type

In [ ]:
resolver.task_id

이제 로컬 컴퓨터에서 두 개의 워커 태스크를 가진 간단한 클러스터를 실행해 보겠습니다. `MultiWorkerMirroredStrategy`을 사용하여 두 태스크로 모델을 훈련하겠습니다.

첫 번째 단계는 훈련 코드를 작성하는 것입니다. 이 코드를 사용해 두 워커에서 각각 고유한 프로세스로 실행하므로 별도의 파이썬 파일인 `my_mnist_multiworker_task.py`에 작성합니다. 코드는 비교적 간단하지만 주의해야 할 몇 가지 중요한 사항이 있습니다:
* 텐서플로로 다른 작업을 수행하기 전에 `MultiWorkerMirroredStrategy`을 생성합니다.
* 워커 중 하나만 텐서보드 로깅을 처리합니다. 앞서 언급했듯이 이 작업자를 *치프*라고 합니다. 명시적으로 정의되지 않은 경우 워커 #0이 치프가 됩니다.

In [ ]:
%%writefile my_mnist_multiworker_task.py

import tempfile
import tensorflow as tf

strategy = tf.distribute.MultiWorkerMirroredStrategy()  # 시작 부분에!
resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()
print(f"Starting task {resolver.task_type} #{resolver.task_id}")

# 추가 코드 - MNIST 데이터셋 로드 및 분할
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape([28, 28, 1], input_shape=[28, 28],
                                dtype=tf.uint8),
        tf.keras.layers.Rescaling(scale=1 / 255),
        tf.keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
                               padding="same", input_shape=[28, 28, 1]),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                               padding="same"),
        tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                               padding="same"),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=64, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=10, activation="softmax"),
    ])
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
                  metrics=["accuracy"])

model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10)

if resolver.task_id == 0:  # 치프는 모델을 올바른 위치에 저장합니다.
    model.save("my_mnist_multiworker_model", save_format="tf")
else:
    tmpdir = tempfile.mkdtemp()  # 다른 워커는 임시 디렉터리에 저장합니다.
    model.save(tmpdir, save_format="tf")
    tf.io.gfile.rmtree(tmpdir)  # 마지막에 이 디렉터리를 삭제할 수 있습니다!

실제 애플리케이션에서는 일반적으로 머신당 하나의 워커가 있지만, 이 예제에서는 동일한 머신에서 두 워커를 모두 실행하고 있으므로 두 워커 모두 사용 가능한 모든 GPU RAM(이 머신에 GPU가 있는 경우)을 사용하려고 시도하므로 메모리 부족(OOM) 오류가 발생할 수 있습니다. 이를 방지하기 위해 `CUDA_VISIBLE_DEVICES` 환경 변수를 사용하여 각 워커에 다른 GPU를 할당할 수 있습니다. 또는 `CUDA_VISIBLE_DEVICES`를 빈 문자열로 설정하여 간단히 GPU 지원을 비활성화할 수 있습니다.

이제 각각 고유한 프로세스에서 두 워커를 시작할 준비가 되었습니다. 태스크 인덱스가 변경된 것을 확인할 수 있습니다:

In [ ]:
%%bash --bg

export CUDA_VISIBLE_DEVICES=''
export TF_CONFIG='{"cluster": {"worker": ["127.0.0.1:9901", "127.0.0.1:9902"]},
                   "task": {"type": "worker", "index": 0}}'
python my_mnist_multiworker_task.py > my_worker_0.log 2>&1

In [ ]:
%%bash --bg

export CUDA_VISIBLE_DEVICES=''
export TF_CONFIG='{"cluster": {"worker": ["127.0.0.1:9901", "127.0.0.1:9902"]},
                   "task": {"type": "worker", "index": 1}}'
python my_mnist_multiworker_task.py > my_worker_1.log 2>&1

**참고**: `AutoShardPolicy`에 대한 경고가 표시되면 무시해도 무방합니다. 자세한 내용은 [TF 이슈 #42146](https://github.com/tensorflow/tensorflow/issues/42146)을 참고하세요.

끝났습니다! 이제 텐서플로 클러스터가 실행 중이지만 별도의 프로세스에서 실행 중이므로 이 노트북에서는 볼 수 없습니다(하지만 진행 상황은 `my_worker_*.log`에서 확인할 수 있습니다).

치프(워커 #0)가 텐서보드에 로그를 쓰기 때문에, 훈련 진행 상황을 보기 위해 텐서보드를 사용할 수 있습니다. 다음 셀을 실행한 다음, 설정 버튼(즉, 톱니바퀴 아이콘)을 클릭하고 "Reload data" 상자를 체크하여 텐서보드가 30초마다 자동으로 새로고침되도록 설정합니다. 첫 번째 훈련이 완료되고(몇 분 정도 소요될 수 있음) 텐서보드가 새로 고침되면 SCALARS 탭이 나타납니다. 이 탭을 클릭하면 모델의 훈련 진행 과정과 검증 정확도를 확인할 수 있습니다.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./my_mnist_multiworker_logs --port=6006

In [ ]:
# strategy = tf.distribute.MultiWorkerMirroredStrategy(
#     communication_options=tf.distribute.experimental.CommunicationOptions(
#         implementation=tf.distribute.experimental.CollectiveCommunication.NCCL))

## 버텍스 AI에서 대규모 훈련 작업 실행하기

훈련 스크립트를 복사하되 `import os`를 추가하고 저장 경로를 `AIP_MODEL_DIR` 환경 변수가 가리키는 GCS 경로로 변경해 보겠습니다:

In [ ]:
%%writefile my_vertex_ai_training_task.py

import os
from pathlib import Path
import tempfile
import tensorflow as tf

strategy = tf.distribute.MultiWorkerMirroredStrategy()  # 시작 부분에!
resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()

if resolver.task_type == "chief":
    model_dir = os.getenv("AIP_MODEL_DIR")  # 버텍스 AI가 제공하는 경로
    tensorboard_log_dir = os.getenv("AIP_TENSORBOARD_LOG_DIR")
    checkpoint_dir = os.getenv("AIP_CHECKPOINT_DIR")
else:
    tmp_dir = Path(tempfile.mkdtemp())  # 다른 워커는 임시 디렉터리를 사용합니다.
    model_dir = tmp_dir / "model"
    tensorboard_log_dir = tmp_dir / "logs"
    checkpoint_dir = tmp_dir / "ckpt"

callbacks = [tf.keras.callbacks.TensorBoard(tensorboard_log_dir),
             tf.keras.callbacks.ModelCheckpoint(checkpoint_dir)]

# 추가 코드 - MNIST 데이터셋 로드 및 준비
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# 추가 코드 - 분산 전략을 사용하여 케라스 모델을 빌드하고 컴파일합니다.
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape([28, 28, 1], input_shape=[28, 28],
                                dtype=tf.uint8),
        tf.keras.layers.Lambda(lambda X: X / 255),
        tf.keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
                               padding="same", input_shape=[28, 28, 1]),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                               padding="same"),
        tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                               padding="same"),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=64, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=10, activation="softmax"),
    ])
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
                  metrics=["accuracy"])

model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10,
          callbacks=callbacks)
model.save(model_dir, save_format="tf")

In [ ]:
custom_training_job = aiplatform.CustomTrainingJob(
    display_name="my_custom_training_job",
    script_path="my_vertex_ai_training_task.py",
    container_uri="gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest",
    model_serving_container_image_uri=server_image,
    requirements=["gcsfs==2022.3.0"],  # 필요 없음, 이것은 단지 예일 뿐입니다.
    staging_bucket=f"gs://{bucket_name}/staging"
)

In [ ]:
mnist_model2 = custom_training_job.run(
    machine_type="n1-standard-4",
    replica_count=2,
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=2,
)

정리합니다:

In [ ]:
mnist_model2.delete()
custom_training_job.delete()
blobs = bucket.list_blobs(prefix=f"gs://{bucket_name}/staging/")
for blob in blobs:
    blob.delete()

## 버텍스 AI의 하이퍼파라미터 튜닝

In [ ]:
%%writefile my_vertex_ai_trial.py

import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--n_hidden", type=int, default=2)
parser.add_argument("--n_neurons", type=int, default=256)
parser.add_argument("--learning_rate", type=float, default=1e-2)
parser.add_argument("--optimizer", default="adam")
args = parser.parse_args()

import tensorflow as tf

def build_model(args):
    with tf.distribute.MirroredStrategy().scope():
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8))
        for _ in range(args.n_hidden):
            model.add(tf.keras.layers.Dense(args.n_neurons, activation="relu"))
        model.add(tf.keras.layers.Dense(10, activation="softmax"))
        opt = tf.keras.optimizers.get(args.optimizer)
        opt.learning_rate = args.learning_rate
        model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
                      metrics=["accuracy"])
        return model

# 추가 코드 - 데이터셋 로드 및 분할
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# 추가 코드 - AIP_* 환경 변수를 사용하고 콜백을 만듭니다.
import os
model_dir = os.getenv("AIP_MODEL_DIR")
tensorboard_log_dir = os.getenv("AIP_TENSORBOARD_LOG_DIR")
checkpoint_dir = os.getenv("AIP_CHECKPOINT_DIR")
trial_id = os.getenv("CLOUD_ML_TRIAL_ID")
tensorboard_cb = tf.keras.callbacks.TensorBoard(tensorboard_log_dir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5)
callbacks = [tensorboard_cb, early_stopping_cb]

model = build_model(args)
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
                    epochs=10, callbacks=callbacks)
model.save(model_dir, save_format="tf")  # 추가 코드

import hypertune

hypertune = hypertune.HyperTune()
hypertune.report_hyperparameter_tuning_metric(
    hyperparameter_metric_tag="accuracy",  # 보고할 지표의 이름
    metric_value=max(history.history["val_accuracy"]),  # 최대 정확도 값
    global_step=model.optimizer.iterations.numpy(),
)

In [ ]:
trial_job = aiplatform.CustomJob.from_local_script(
    display_name="my_search_trial_job",
    script_path="my_vertex_ai_trial.py",  # 훈련 스크립트 경로
    container_uri="gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest",
    staging_bucket=f"gs://{bucket_name}/staging",
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=2,  # 이 예제에서는 각 트라이얼에 2개의 GPU가 있습니다.
)

In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

hp_job = aiplatform.HyperparameterTuningJob(
    display_name="my_hp_search_job",
    custom_job=trial_job,
    metric_spec={"accuracy": "maximize"},
    parameter_spec={
        "learning_rate": hpt.DoubleParameterSpec(min=1e-3, max=10, scale="log"),
        "n_neurons": hpt.IntegerParameterSpec(min=1, max=300, scale="linear"),
        "n_hidden": hpt.IntegerParameterSpec(min=1, max=10, scale="linear"),
        "optimizer": hpt.CategoricalParameterSpec(["sgd", "adam"]),
    },
    max_trial_count=100,
    parallel_trial_count=20,
)
hp_job.run()

In [ ]:
def get_final_metric(trial, metric_id):
    for metric in trial.final_measurement.metrics:
        if metric.metric_id == metric_id:
            return metric.value

trials = hp_job.trials
trial_accuracies = [get_final_metric(trial, "accuracy") for trial in trials]
best_trial = trials[np.argmax(trial_accuracies)]

In [ ]:
max(trial_accuracies)

In [ ]:
best_trial.id

In [ ]:
best_trial.parameters

# 추가 자료 - 버텍스 AI의 분산 케라스 튜너

버텍스 AI의 하이퍼파라미터 튜닝 서비스를 사용하는 대신, 10장에서 소개한 [케라스 튜너](https://keras.io/keras_tuner/)를 사용하여 버텍스 AI VM에서 실행할 수 있습니다. 케라스 튜너는 하이퍼파라미터 탐색을 여러 머신에 분산하여 간단하게 확장할 수 있는 방법을 제공합니다: 각 머신에서 세 개의 환경 변수를 설정한 다음 일반 케라스 튜너 코드를 실행하기만 하면 됩니다. 모든 머신에서 똑같은 스크립트를 사용할 수 있습니다. 머신 중 한 대는 치프 역할을 하고 다른 머신은 워커 역할을 합니다. 각 워커는 오라클 역할을 하는 치프에게 어떤 하이퍼파라미터 값을 시도할지 요청하고, 치프는 이 하이퍼파라미터 값을 사용하여 모델을 훈련시킨 다음, 최종적으로 모델의 성능을 치프에게 보고하면 치프는 다음에 워커가 시도할 하이퍼파라미터 값을 결정할 수 있습니다.

각 머신에서 설정해야 하는 세 가지 환경 변수는 다음과 같습니다:

* `KERASTUNER_TUNER_ID`: 치프 머신은 `"chief"`이고 워커 머신은 `"worker0"`, `"worker1"` 등과 같은 고유 식별자입니다.
* `KERASTUNER_ORACLE_IP`: 치프 머신의 IP 주소 또는 호스트 이름입니다. 치프 자체는 일반적으로 `"0.0.0.0"`을 사용하여 머신의 모든 IP 주소에서 수신해야 합니다.
* `KERASTUNER_ORACLE_PORT`: 치프가 수신 대기할 TCP 포트입니다.

분산 케라스 튜너는 모든 머신 집합에서 사용할 수 있습니다. 버텍스 AI 머신에서 실행하려면 일반 훈련 작업을 생성하고 훈련 스크립트를 수정하여 세 가지 환경 변수를 적절히 설정한 후 케라스 튜너를 사용하면 됩니다.

예를 들어, 아래 스크립트는 이전과 마찬가지로 먼저 버텍스 AI가 자동으로 설정하는 `TF_CONFIG` 환경 변수를 파싱합니다. `"chief"` 타입의 태스크 주소를 찾아서 IP 주소 또는 호스트 이름과 TCP 포트를 추출합니다. 그런 다음 튜너 ID를 태스크 타입과 태스크 인덱스로 정의합니다(예: `"worker0"`). 튜너 ID가 `"chief0"`이면 `"chief"`로 변경하고 IP를 `"0.0.0.0"`으로 설정하면 해당 컴퓨터의 모든 IPv4 주소에서 수신 대기하게 됩니다. 그런 다음 케라스 튜너에 대한 환경 변수를 정의합니다. 다음으로 스크립트는 10장에서와 마찬가지로 튜너를 생성하여 탐색을 수행한 다음 마지막으로 버텍스 AI가 지정한 위치에 최적의 모델을 저장합니다:

In [ ]:
%%writefile my_keras_tuner_search.py

import json
import os

tf_config = json.loads(os.environ["TF_CONFIG"])

chief_ip, chief_port = tf_config["cluster"]["chief"][0].rsplit(":", 1)
tuner_id = f'{tf_config["task"]["type"]}{tf_config["task"]["index"]}'
if tuner_id == "chief0":
    tuner_id = "chief"
    chief_ip = "0.0.0.0"
    # 추가 코드 - 치프는 많은 작업을 수행하지 않으므로 동일한 컴퓨터에서 워커를 실행하여
    # 컴퓨팅 리소스를 최적화할 수 있습니다. 이렇게 하려면
    # TF_CONFIG 환경 변수를 조정하여 태스크 유형을 "worker"로 설정하고 태스크 인덱스를
    # 고유한 값으로 설정한 후 치프가 또 다른 프로세스를 시작하도록 하면 됩니다.
    # 다음 몇 줄의 주석 처리를 해제하고 시도해 보세요:
    # import subprocess
    # import sys
    # tf_config["task"]["type"] = "workerX"  # 치프 머신의 워커
    # os.environ["TF_CONFIG"] = json.dumps(tf_config)
    # subprocess.Popen([sys.executable] + sys.argv,
    #                  stdout=sys.stdout, stderr=sys.stderr)

os.environ["KERASTUNER_TUNER_ID"] = tuner_id
os.environ["KERASTUNER_ORACLE_IP"] = chief_ip
os.environ["KERASTUNER_ORACLE_PORT"] = chief_port

from pathlib import Path
import keras_tuner as kt
import tensorflow as tf

gcs_path = "/gcs/my_bucket/my_hp_search"  # 버킷 이름으로 바꾸기

def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
                             sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8))
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=optimizer,
                  metrics=["accuracy"])
    return model

hyperband_tuner = kt.Hyperband(
    build_model, objective="val_accuracy", seed=42,
    max_epochs=10, factor=3, hyperband_iterations=2,
    distribution_strategy=tf.distribute.MirroredStrategy(),
    directory=gcs_path, project_name="mnist")

# extra code – Load and split the MNIST dataset
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

tensorboard_log_dir = os.environ["AIP_TENSORBOARD_LOG_DIR"] + "/" + tuner_id
tensorboard_cb = tf.keras.callbacks.TensorBoard(tensorboard_log_dir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5)
hyperband_tuner.search(X_train, y_train, epochs=10,
                       validation_data=(X_valid, y_valid),
                       callbacks=[tensorboard_cb, early_stopping_cb])

if tuner_id == "chief":
    best_hp = hyperband_tuner.get_best_hyperparameters()[0]
    best_model = hyperband_tuner.hypermodel.build(best_hp)
    best_model.save(os.getenv("AIP_MODEL_DIR"), save_format="tf")

참고로 버텍스 AI는 오픈 소스 [GCS 퓨즈 어댑터](https://cloud.google.com/storage/docs/gcs-fuse)를 사용하여 `/gcs` 디렉터리를 GCS에 자동으로 마운트합니다. 이렇게 하면 워커와 치프 간에 공유 디렉터리가 제공되며, 이는 케라스 튜너에 필요합니다. 또한 배포 전략을 `MirroredStrategy`으로 설정했습니다. 이렇게 하면 각 워커가 자신의 머신에 있는 모든 GPU를 사용할 수 있습니다(GPU가 두 개 이상 있는 경우).

`gcs/my_bucket/`를 <code>/gcs/<i>{bucket_name}</i>/</code>로 바꿉니다:

In [ ]:
with open("my_keras_tuner_search.py") as f:
    script = f.read()

with open("my_keras_tuner_search.py", "w") as f:
    f.write(script.replace("/gcs/my_bucket/", f"/gcs/{bucket_name}/"))

이제 이전 섹션에서와 똑같이 이 스크립트를 기반으로 사용자 정의 훈련 작업을 시작하기만 하면 됩니다. `requirements` 목록에 `keras-tuner`를 추가하는 것을 잊지 마세요:

In [ ]:
hp_search_job = aiplatform.CustomTrainingJob(
    display_name="my_hp_search_job",
    script_path="my_keras_tuner_search.py",
    container_uri="gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest",
    model_serving_container_image_uri=server_image,
    requirements=["keras-tuner~=1.1.2"],
    staging_bucket=f"gs://{bucket_name}/staging",
)

In [ ]:
mnist_model3 = hp_search_job.run(
    machine_type="n1-standard-4",
    replica_count=3,
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=2,
)

모델을 찾았습니다!

정리:

In [ ]:
mnist_model3.delete()
hp_search_job.delete()
blobs = bucket.list_blobs(prefix=f"gs://{bucket_name}/staging/")
for blob in blobs:
    blob.delete()

# 추가 자료 - AutoML을 사용하여 모델 훈련하기

먼저 MNIST 데이터셋을 PNG 이미지로 내보내고, 각 이미지 파일과 분할(훈련, 검증 또는 테스트), 레이블을 나타내는 `import.csv`를 준비해 보겠습니다:

In [ ]:
import matplotlib.pyplot as plt

mnist_path = Path("datasets/mnist")
mnist_path.mkdir(parents=True, exist_ok=True)
idx = 0
with open(mnist_path / "import.csv", "w") as import_csv:
    for split, X, y in zip(("training", "validation", "test"),
                           (X_train, X_valid, X_test),
                           (y_train, y_valid, y_test)):
        for image, label in zip(X, y):
            print(f"\r{idx + 1}/70000", end="")
            filename = f"{idx:05d}.png"
            plt.imsave(mnist_path / filename, np.tile(image, 3))
            line = f"{split},gs://{bucket_name}/mnist/{filename},{label}\n"
            import_csv.write(line)
            idx += 1

이 데이터셋을 GCS에 업로드해 보겠습니다:

In [ ]:
upload_directory(bucket, mnist_path)

이제 버텍스 AI에서 관리용 이미지 데이터셋을 만들어 보겠습니다:

In [ ]:
from aiplatform.schema.dataset.ioformat.image import single_label_classification

mnist_dataset = aiplatform.ImageDataset.create(
    display_name="mnist-dataset",
    gcs_source=[f"gs://{bucket_name}/mnist/import.csv"],
    project=project_id,
    import_schema_uri=single_label_classification,
    sync=True,
)

이 데이터셋에 대해 AutoML 학습 작업을 생성합니다:

**TODO**

# 연습문제 해답

## 1. to 8.

부록 A 참조

## 9.
_문제: (원하는 어떤 모델이든) 모델을 훈련하고 TF 서빙이나 구글 버텍스 AI 플랫폼에 배포해보세요. REST API나 gRPC API를 사용해 쿼리하는 클라이언트 코드를 작성해보세요. 모델을 업데이트하고 새로운 버전을 배포해보세요. 클라이언트 코드가 새로운 버전으로 쿼리할 것입니다. 첫 번째 버전으로 롤백해보세요._

<a href="#텐서플로-모델-서빙하기">텐서플로 모델 서빙하기</a> 절에 있는 단계를 따라해 보세요.

# 10.
_문제: 하나의 머신에 여러 개의 GPU에서 `MirroredStrategy` 전략으로 모델을 훈련해보세요(GPU를 준비하지 못하면 코랩의 GPU 런타임을 사용하여 가상 GPU 2개를 만들 수 있습니다). `CentralStorageStrategy` 전략으로 모델을 다시 훈련하고 훈련 시간을 비교해보세요._

[여러 디바이스에서 모델 훈련하기](#여러-디바이스에서-모델-훈련하기) 절에 있는 단계를 따라해 보세요.

# 11.
_문제: 케라스 튜너 또는 버텍스 AI의 하이퍼파라미터 튜닝 서비스를 사용하여 버텍스 AI에서 원하는 모델을 세부 튜닝해 보세요._

이 책의 _케라스 튜너를 사용한 하이퍼파라미터 튜닝_ 섹션에 있는 지침을 따르세요.

# 축하합니다!

책의 마지막에 도달했습니다! 도움이 되셨기를 바랍니다. 😊